In [13]:
import numpy as np
import time
import pandas as pd
import json

print(time.strftime("%H:%M:%S", time.gmtime()))

# Import amazon reviews dataset 

filepath = './Clothing_Shoes_and_Jewelry_5.json'

data = []
with open(filepath
            , 'r') as f:
        for line in f:
            data.append(json.loads(line))

df = pd.DataFrame(data)


df["length_review"] = df["reviewText"].apply(lambda x: len(x.split()))
df["length_summary"] = df["summary"].apply(lambda x: len(x.split()))

print(df["reviewText"][0])
df.head()

# SUbset 10000 reviews for faster processing
df2  = df.sample(10000)
df2.shape




04:12:33
This is a great tutu and at a really great price. It doesn't look cheap at all. I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly. A++


(10000, 11)

In [14]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nanchen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [16]:
# Run Vader on the reviews
analyzer = SentimentIntensityAnalyzer()

def vader_polarity(text):
    return analyzer.polarity_scores(text)['compound']
# Calculate the breakdown of the sentiment
def vader_positive(text):
    return analyzer.polarity_scores(text)['pos']
def vader_neutral(text):
    return analyzer.polarity_scores(text)['neu']
def vader_negative(text):
    return analyzer.polarity_scores(text)['neg']

# Calculate the sum of all the lexicon ratings and takes values from -1 to 1. 
# The closer to 1, the more positive the text is.
df2['vader_polarity'] = df2['reviewText'].apply(vader_polarity)
# the proportion of the text that falls into positive, neutral, and negative categories, respectively
df2['vader_positive'] = df2['reviewText'].apply(vader_positive)
df2['vader_neutral'] = df2['reviewText'].apply(vader_neutral)
df2['vader_negative'] = df2['reviewText'].apply(vader_negative)

# textblob
def textblob_polarity(text):
    return TextBlob(text).sentiment.polarity
def textblob_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Returns a polarity score between -1 and 1 where 1 means positive statement and -1 means a negative statement.
df2['textblob_polarity'] = df2['reviewText'].apply(textblob_polarity)
# A subjectivity score between 0 and 1. Subjectivity quantifies the amount of personal 
# opinion and factual information contained in the text. The higher subjectivity means that 
# the text contains personal opinion rather than factual information.
df2['textblob_subjectivity'] = df2['reviewText'].apply(textblob_subjectivity)

df2.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,length_review,length_summary,vader_polarity,vader_positive,vader_neutral,vader_negative,textblob_polarity,textblob_subjectivity
92548,AIFPCDUD7B14B,B002Z7EOGW,susan,"[0, 0]","These arrived quickly, and fit perfectly! My h...",5.0,mens Cudas Water Shoe,1390176000,"01 20, 2014",25,4,0.9244,0.401,0.599,0.000,0.361905,0.632143
190510,A1TVGGVRMO5LMZ,B007L6GBZ4,Sandy,"[0, 0]",Bought for my son and I think for short term t...,4.0,ok,1369872000,"05 30, 2013",36,1,0.8338,0.228,0.772,0.000,0.225000,0.427778
172026,ATOSQ6RK52IX7,B006DU3EM2,doctor,"[0, 0]","I read all reviews for this product, and I ord...",3.0,Converse Chuck Taylor All Star Shoes (M9166) L...,1380758400,"10 3, 2013",65,14,0.4939,0.115,0.851,0.034,0.025000,0.415000
13822,A13ITPS2P2ZRK6,B00085FIPO,Shannon R.,"[0, 0]",I was having such a hard time finding shorts t...,4.0,Great product for the money,1339718400,"06 15, 2012",95,5,0.5023,0.059,0.928,0.013,0.133712,0.389286
129289,A2G66KTJIE9FI4,B004IYZKWW,Debby,"[1, 1]",I wear a size 7 and these were too short and t...,2.0,smaller than size shown,1341014400,"06 30, 2012",48,4,0.7184,0.125,0.875,0.000,0.094898,0.351020


In [17]:
# run correlation between vader and textblob
# A correlation matrix that shows the correlation coefficients between these three variables.
# Correlation coefficients range from -1 to 1. A value close to 1 implies a strong positive 
# correlation (as one variable increases, the other tends to increase), a value close to -1 
# implies a strong negative correlation (as one variable increases, the other tends to decrease), 
# and a value close to 0 implies no correlation.
#  if ‘vader_polarity’ and ‘textblob_polarity’ have a high positive correlation, 
# it means that both methods tend to agree on the sentiment of the reviews. 
# This could be useful in validating the consistency of the sentiment analysis methods used.
df2[['vader_polarity', 'textblob_polarity', 'textblob_subjectivity']].corr()

,vader_polarity,textblob_polarity,textblob_subjectivity
vader_polarity,1.000000,0.48244,0.207044
textblob_polarity,0.482440,1.00000,0.451960
textblob_subjectivity,0.207044,0.45196,1.000000


Vader_polarity and textblob_polarity correlation is approximately 0.48244. This indicates a moderate positive correlation between the sentiment scores calculated by VADER and TextBlob. In other words, when the sentiment score from VADER increases, the sentiment score from TextBlob also tends to increase, but not as strongly.

Vader_polarity and textblob_subjectivity correlation is approximately 0.207044. This indicates a weak positive correlation. This means that there’s a slight tendency that as the sentiment score from VADER increases, the subjectivity score from TextBlob also increases, but this relationship is not very strong.

Textblob_polarity and textblob_subjectivity correlation is approximately 0.45196. This indicates a moderate positive correlation. This means that there’s a moderate tendency that as the sentiment score from TextBlob increases, the subjectivity score from TextBlob also increases.

The above result shows how these sentiment and subjectivity scores relate to each other. For example, the moderate positive correlation between vader_polarity and textblob_polarity suggests that these two methods are somewhat consistent in their sentiment scoring. However, the weak correlation between vader_polarity and textblob_subjectivity suggests that the sentiment score from VADER doesn’t strongly relate to the subjectivity score from TextBlob.

In [18]:

# Run ANOVA to see if the polarity is different for different ratings
# determine whether there are any statistically significant differences between the means of three or more independent groups
from scipy import stats

stats.f_oneway(df2['vader_polarity'][df2['overall'] == 1],
                df2['vader_polarity'][df2['overall'] == 2],
                df2['vader_polarity'][df2['overall'] == 3],
                df2['vader_polarity'][df2['overall'] == 4],
                df2['vader_polarity'][df2['overall'] == 5])

stats.f_oneway(df2['textblob_polarity'][df2['overall'] == 1],
                df2['textblob_polarity'][df2['overall'] == 2],
                df2['textblob_polarity'][df2['overall'] == 3],
                df2['textblob_polarity'][df2['overall'] == 4],
                df2['textblob_polarity'][df2['overall'] == 5])

F_onewayResult(statistic=565.0513472950851, pvalue=0.0)

The ANOVA test is showing that there is a statistically significant difference in the sentiment scores (both VADER and TextBlob) between the different rating groups.

statistic=565.0513472950851: This is the F-statistic, a value you get when you run an ANOVA test. It represents the ratio of the variation between the group means to the variation within the groups. The larger this value is, the more likely it is that the differences between the group means are statistically significant.

pvalue=0.0: The p-value is the probability of getting a result at least as extreme as the one that was actually observed, given that the null hypothesis is true. In this case, the null hypothesis is that there’s no difference in the mean sentiment scores between the different rating groups. A p-value of 0.0 is indicating that the likelihood of the observed differences arising by chance (assuming the null hypothesis is true) is extremely low. Therefore, you would reject the null hypothesis and conclude that there is a statistically significant difference in the mean sentiment scores between the different rating groups.

So, in this case, the ANOVA test results with an F-statistic of 565.0513472950851 and a p-value of 0.0 indicate a statistically significant difference in polarity scores across different overall ratings for both VADER and TextBlob. This suggests that the sentiment expressed in reviews varies significantly with the star ratings given by customers.


In [19]:
# Describe() polarity scores by rating
df2.groupby('overall').describe()[['vader_polarity', 'textblob_polarity']]

vader_polarity                                                       \
                 count      mean       std     min      25%     50%     75%   
overall                                                                       
1.0              385.0 -0.004888  0.601752 -0.9839 -0.57090  0.0000  0.5499   
2.0              532.0  0.200063  0.574307 -0.9720 -0.25385  0.2830  0.7163   
3.0             1111.0  0.417149  0.512730 -0.9318  0.05185  0.5859  0.8428   
4.0             2095.0  0.689848  0.361603 -0.9085  0.59420  0.8402  0.9278   
5.0             5877.0  0.796263  0.274763 -0.8873  0.77170  0.8940  0.9501   

                textblob_polarity                                          \
            max             count      mean       std       min       25%   
overall                                                                     
1.0      0.9869             385.0  0.006029  0.219880 -0.750000 -0.103571   
2.0      0.9926             532.0  0.079853  0.175410 -0.766667 -0.017770   
3.0      0.9973            1111.0  0.150210  0.189286 -0.780000  0.035635   
4.0      0.9993            2095.0  0.228330  0.173584 -0.400000  0.115994   
5.0      0.9995            5877.0  0.325113  0.196590 -0.875000  0.192262   

                                       
              50%       75%       max  
overall                                
1.0      0.025002  0.136667  0.737500  
2.0      0.087727  0.180864  0.633333  
3.0      0.138095  0.254356  1.000000  
4.0      0.208333  0.333333  1.000000  
5.0      0.310571  0.447222  1.000000

The descriptive statistics for VADER and TextBlob polarity scores, grouped by overall ratings, provide further insights:

The columns “min”, “25%”, “50%”, “75%”, and “max” represent different percentiles and extremes in the distribution of polarity scores. These statistics help in sentiment analysis tasks by providing a comprehensive view of how sentiments are distributed across different ratings

Min: The lowest polarity score, indicating the most negative sentiment within the group.
25%: The lower quartile, where 25% of scores are below this value, showing the lower bound of sentiment scores.
50%: The median, indicating the middle sentiment score.
75%: The upper quartile, where 75% of scores are below this value, showing the upper bound of sentiment scores.
Max: The highest polarity score, indicating the most positive sentiment within the group.

1-Star Ratings: Negative mean polarity scores from both VADER and TextBlob suggest that reviews with 1-star ratings express negative sentiments.

2 to 5-Star Ratings: Positive mean polarity scores indicate that reviews with higher star ratings express more positive sentiments. This trend is consistent across both VADER and TextBlob, with the mean polarity scores increasing as the star ratings increase. Correspond to higher positive mean polarities, indicating more positive sentiments.

Standard Deviation: The standard deviation for VADER polarity scores tends to decrease with higher ratings, suggesting more consistency in positive sentiment expression among higher-rated reviews. TextBlob’s standard deviation does not show a consistent trend.


These results confirm that sentiment scores are correlated with the star ratings given by customers. For instance, a high VADER polarity score is likely to correspond to a high star rating, which can be used to predict customer satisfaction levels. Similarly, TextBlob’s polarity and subjectivity scores can provide additional layers of understanding about the reviews’ sentiments and how much personal opinion they contain.

